### ResNet50V2

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
Xtrain = np.load('Xtrain.npz')
Xtrain = Xtrain['x']
Ytrain = np.load('Ytrain.npz')
Ytrain = Ytrain['y']

In [3]:
Xtest = np.load('Xtest.npz')
Xtest = Xtest['x']
Ytest = np.load('Ytest.npz')
Ytest = Ytest['y']

In [4]:
Xval = np.load('Xval.npz')
Xval = Xval['x']
Yval = np.load('Yval.npz')
Yval = Yval['y']

In [5]:
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(Xtrain.shape[1], Xtrain.shape[2], 3))

In [6]:
Xtrain = np.expand_dims(Xtrain, axis=-1) 
Xtrain = np.repeat(Xtrain, 3, axis=-1)

In [7]:
Xval = np.expand_dims(Xval, axis=-1)
Xval = np.repeat(Xval, 3, axis=-1)

In [15]:
Xtest = np.expand_dims(Xtest, axis=-1)
Xtest = np.repeat(Xtest, 3, axis=-1)

In [8]:
base_model.trainable = False

In [9]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')  # Assuming num_classes is the number of classes you want to classify
])

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

In [12]:
train_generator = train_datagen.flow(Xtrain, Ytrain, batch_size=32)
validation_data = (Xval, Yval)
model.fit(train_generator, epochs=10, validation_data=validation_data)

Epoch 1/10
92/92 [==============================] - 226s 2s/step - loss: 0.9465 - accuracy: 0.6509 - val_loss: 231.2335 - val_accuracy: 0.2842
Epoch 2/10
92/92 [==============================] - 268s 3s/step - loss: 0.8045 - accuracy: 0.7003 - val_loss: 226.8943 - val_accuracy: 0.2842
Epoch 3/10
92/92 [==============================] - 264s 3s/step - loss: 0.7807 - accuracy: 0.7099 - val_loss: 268.3937 - val_accuracy: 0.2842
Epoch 4/10
92/92 [==============================] - 266s 3s/step - loss: 0.7347 - accuracy: 0.7263 - val_loss: 321.5524 - val_accuracy: 0.2842
Epoch 5/10
92/92 [==============================] - 265s 3s/step - loss: 0.7391 - accuracy: 0.7239 - val_loss: 161.7446 - val_accuracy: 0.2814
Epoch 6/10
92/92 [==============================] - 230s 2s/step - loss: 0.7139 - accuracy: 0.7365 - val_loss: 163.4162 - val_accuracy: 0.3743
Epoch 7/10
92/92 [==============================] - 232s 3s/step - loss: 0.7068 - accuracy: 0.7348 - val_loss: 184.9222 - val_accuracy: 0.4016

In [16]:
test_loss, test_acc = model.evaluate(Xtest, Ytest)
print('Test accuracy:', test_acc)

12/12 [==============================] - 22s 2s/step - loss: 289.7845 - accuracy: 0.2377
Test accuracy: 0.23770491778850555
